# Noms du binome: CILLEROS JEAN

# Prénoms du binome: Victor Jérémy

## Note: exporter le compte rendu basé sur le notebook au format pdf


# Entropie et génération de mots de passe
* L'objectif de ce tp est de continuer à se familiariser avec la notion d'entropie, mais aussi de comprendre le lien qu'il existe entre cette mesure informationelle et la sécurité d'un générateur (humain ou executable) de mots de passes
* Ainsi, nous proposons d'étudier l'entropie d'un tel générateur, et ce en fonction du modèle probabiliste considéré pour le modéliser (contruit à partir d'une lettre, de deux lettres, de 4 lettres, ...). A l'aide de tirrages aléatoires, nous estimerons également le temps moyen nécessaire pour trouver un mot de passe à partir de ce modèle.
* A la fin de ce TP, nous considérerons un générateur de mots de passe spécifique qui générera un mot de passe en prennant **8 lettres consécutives dans un texte** (sans se soucier des espaces). Ces lettres peuvent faire parti d'un ou de plusieurs mots consécutifs.
* Nous faisons l'hypothèse que le texte n'est composé que des 26 lettres de l'alphabet, sans majuscules ni accents

Nous chercherons aussi à comprendre (voir dernière question):
- les bonnes pratiques pour le défenseur, i.e. la personne cherchant à générer/construire un système de génération de mots de passe.
- les bonnes pratiques pour l'attaquant, i.e. la personne essayant de trouver le mot de passe.

**Il est important de commenter vos réponses, en utilisant des cellules markdown**


In [1]:
import numpy as np

from numpy import genfromtxt
from pandas import read_csv
import pandas as pd
import time

#### Modèle monogramme (une lettre)
* On récupére des données composées de [lettre,frequence d'apparition de la lettre] (voir fichier csv pour [comma-separated-value](https://en.wikipedia.org/wiki/Comma-separated_values)) 

In [2]:
monogramme = read_csv('monogramme.csv')
freq_mono = (monogramme['frequency']).values
letters_mono = (monogramme['letters']).values



Q: Quelles sont les 5 lettres les plus représentées ?

In [3]:
# TO DO
monogramme.sort_values(by='frequency', ascending=False).letters.iloc[:5]

0    E
1    S
2    A
3    N
4    T
Name: letters, dtype: object

Les 5 lettres les plus représentées sont : 
E S A N T

Ecrire une fonction qui calcule l'entropie à partir d'un vecteur constitué de probabilités empiriques (note, il est important de bien "gérer le cas ou la probabilité est nulle).

In [5]:
def entropie(freq):
    ent = 0
    for f in freq:
        if f>0:
            ent -= f*np.log2(f)
    return ent

Q: en utilisant ce modèle probabiliste pour générer un mot de passe, quelle est l'entropie d'un mot de passe de 8 lettres ?

In [5]:
# TO DO
print(f"L'entropie pour un mot de passe de 8 lettres avec cde modèle probabiliste est minimale pour : {round(8*entropie(monogramme['frequency'].tolist()),4)} bits.")

L'entropie pour un mot de passe de 8 lettres avec cde modèle probabiliste est minimale pour : 31.6762 bits.


Q: A l'aide de la fonction `np.random.choice()`, estimer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ? (note: ici le tirage n'est pas forcemment réaliste, car aléatoire, mais l'idée est surtout de mesurer le temps minimal nécessaire pour générer N mots de passes).

In [3]:
# TO DO
t_100000=0
letters = monogramme.letters.tolist()
proba=monogramme.frequency.tolist()
nbEssais=100_000
for __ in range(10):
    t = time.time()
    essai= np.random.choice(letters,p=proba,size=(nbEssais,8))
    t_100000 += time.time() - t
tempsref = t_100000/10
print(f"Le temps nécessaire pour tirer 100 000 mots de passe est : {tempsref} s")

Le temps nécessaire pour tirer 100 000 mots de passe est : 0.028124737739562988 s


Ici calcul de ce temps est une moyenne sur 10 essais 

Nous definissons l'"entropie du devin" G (guessing entropie) comme le **nombre moyen d'essais successif nécessaires pour trouver un mot de passe à partir de notre générateur**.
    On peut montrer que $G\geq 2^H/4+1$ où $H$ est l'entropie de la source (voir le papier Password_Entropy_and_Password_Quality.pdf )

Q: calculer le minorant de $G$ pour ce modèle

In [6]:
# TO DO
H=8*entropie(monogramme['frequency'].tolist())
minG=(2**H/4)+1
print(f"La valeur du minorant est : {int(minG)}")

La valeur du minorant est : 857904864


Q: combien de temps cela prendra-t-il pour générer un mot de passe si l'on suppose qu'il est possible de prendre le générateur codé précédemment ? (en minutes)

In [8]:
# TO DO
print(f"Il faut en moyenne : {((tempsref*minG)/100000)/60} minutes ")

Il faut en moyenne : 3.8153068148042424 minutes 


On propose maintenant d'utiliser un modèle plus évolué qui est construit à partir de la probabilité conjointe de deux lettres successives (bigramme)

In [8]:
bigramme = read_csv('bigramme.csv',keep_default_na=False)
print(bigramme.head(5))
freq_bi = (bigramme['frequency']).values
letters_bi = (bigramme['letters']).values

  letters  frequency
0      AA   0.000252
1      AB   0.001760
2      AC   0.003482
3      AD   0.001960
4      AE   0.000200


Q: Quelles sont les 5 couples de lettres les plus représentés ?

In [9]:
# TO DO
bigramme.sort_values(by='frequency', ascending=False).letters.iloc[:5]

122    ES
117    EN
82     DE
290    LE
357    NT
Name: letters, dtype: object

Les cinq couples les plus représentés sont :
ES - EN - DE - LE - NT

Q: en utilisant ce modèle probabiliste pour générer un mot de passe, quelle est l'entropie d'un mot de passe de 8 lettres ?

In [10]:
print(f"L'entropie pour un mot de passe de 8 lettres avec cde modèle probabiliste est minimale pour : {round(4*entropie(bigramme['frequency'].tolist()),4)} bits.")

L'entropie pour un mot de passe de 8 lettres avec cde modèle probabiliste est minimale pour : 30.1423 bits.


Q: Pourquoi cette entropie est-elle inférieure à celle du modèle construit sur des monogrammes ? Quelle propriété théorique de l'entropie peut justifier ce constat  ?

D'après le cours (entropie conditionnelle) on a : $H(X,Y) <= H(X) + H(Y)$ 

Ainsi on a $H(X,X) <= 2*H(X)$

Nécessairement $4*H(X,X) <= 8*H(X)$

Q: A l'aide de la fonction `np.random.choice()`, calculer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ?

In [11]:
# TO DO
t_100000=0
letters = bigramme.letters.tolist()
proba=bigramme.frequency.tolist()
nbEssais=100_000
for __ in range(10):
    t = time.time()
    essai= np.random.choice(letters,p=proba,size=(nbEssais,4))
    t_100000 += time.time() - t
tempsrefBigramme = t_100000/10
print(f"Le temps nécessaire pour tirer 100 000 mots de passe est : {tempsrefBigramme} s")

Le temps nécessaire pour tirer 100 000 mots de passe est : 0.02622983455657959 s


Q: calculer le minorant de $G$ pour ce modèle

In [13]:
# TO DO
HBigramme=4*entropie(bigramme['frequency'].tolist())
minGBigramme=(2**HBigramme/4)+1
print(f"La valeur du minorant est : {int(minGBigramme)}")

La valeur du minorant est : 296254956


Q: combien de temps cela prendra-t-il si l'on suppose qu'il est possible de prendre le générateur codé précédemment ? (en minutes)

In [14]:
print(f"Il faut en moyenne : {((tempsrefBigramme*minGBigramme)/100000)/60} minutes ")

Il faut en moyenne : 1.2951197501413454 minutes 


Q: si maintenant on change de stratégie et on tire aléatoirement chaque lettre de l'alphabet de façon uniforme, quelle est l'entropie de ce nouveau générateur ?

In [16]:
H_unif = 8* np.log2(26)
print(f"Entropie maximale car cas uniforme : {H_unif} bit")

Entropie maximale car cas uniforme : 37.603517745128734 bit


Q: A l'aide de la fonction `np.random.choice()`, calculer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ?

In [19]:
# TO DO
t_100000=0
letters = monogramme.letters.tolist()
probaUnif=[1/26 for i in range(len(letters))]
nbEssais=100_000
for __ in range(10):
    t = time.time()
    essai= np.random.choice(letters,p=probaUnif,size=(nbEssais,12))
    t_100000 += time.time() - t
tempsrefUnif = t_100000/10
print(f"Le temps nécessaire pour tirer 100 000 mots de passe est : {tempsrefUnif} s")

Le temps nécessaire pour tirer 100 000 mots de passe est : 0.047572779655456546 s


Q: calculer le minorant de $G$ pour ce modèle

In [20]:
HUnif=8*entropie(probaUnif)
minGUnif=(2**HUnif/4)+1
print(f"La valeur du minorant est : {int(minGUnif)}")

La valeur du minorant est : 52206766144


Q: dans ce cas précis, quelle est la valeur exacte de $G$?

La valur exacte de G se calcule dans ce cas avec l'espérance du nombre de coups avant de tomber sur le mot de passe :
$ E(X=k) = \sum_{k=0}^{26^{8}}P(X=k)k $
Avec $P(X=k) = \frac{1}{26^{8}} $
D'ou $E(X) = G = \frac{26^{8} + 1}{2}$

Q: combien de temps cela prendra-t-il en utilisant le générateur codé précédemment ? (en minutes)

In [21]:
GexactUnif = ((26**8)+1)/2
print(f"Il faut en moyenne : {((tempsrefUnif*GexactUnif)/100000)/60} minutes ")

Il faut en moyenne : 827.873660768118 minutes 


Q: implémenter une attaque pratique qui consiste à:
1. **pour le défenseur:** tirer un mot de passe de 4 lettres consécutives à partir de ce texte de Victor Hugo (texteFrancais.txt) tiré des Misérables.  
2. **pour l'attaquant:** utiliser le modèle bigramme pour générer des mots de passe et minimiser le nombre d'essais. Pour cela on pourra :
    * dans un premier temps pré-calculer un **dictionnaire**, qui contriendra un nombre de MdP générés classés dans l'ordre du plus probable au moins probable et qui ne contient pas de doublons 
    * dans un deuxième temps appeler ce dictionnaire pour comparer chacune de ses entrées au mot de passe généré.
3. Il faudra faire ses tests plusieurs fois afin de d'obtenir un nombre moyens d'appel au dictionnaire nécessaire
4. Il sera intéressant de comparer le nombre trouvé à la valeur de G (qui est une borne inférieure)
5. Question annexe: Par un simple calcul, si le générateur utilisé n'est plus ce générateur mais un générateur qui tire chaque lettre de façon équiprobable, rappeler la valeur de G. Comparer cette valeur avec la valeur trouvée en utilisant la stratégie "des 4 lettres consécutives".

In [20]:
from unidecode import unidecode
import re
pattern = r'[^A-Za-z0-9]+'
## Fonction générant un mot de passe
def get_passwd():
    text_hugo = open("texteFrancais.txt","r")
    str_hugo = str(text_hugo.read())

    # On remplace des lettres avec accent avec des lettres sans accent
    str_hugo = str_hugo.replace("Â", "A")
    str_hugo = str_hugo.replace("Ù", "U")
    str_hugo = str_hugo.replace("Ô", "O")
    str_hugo = unidecode(str_hugo)
    str_hugo = re.sub(pattern, '', str_hugo)
    size_txt = len(str_hugo)

    idx_rand = np.random.randint(size_txt-4)
    #print(idx_rand)

    psswd = str_hugo[idx_rand:idx_rand+4].upper()
    return(psswd)

In [35]:
import itertools

def generate_dictionary():
    mdp = []
    proba = []
    couples = list(itertools.combinations(bigramme.letters.tolist(),2))
    probs = list(itertools.combinations(bigramme.frequency.tolist(),2))
    for c, p in zip(couples,probs):
        mdp.append(c[0] + c[1])
        mdp.append(c[1] + c[0])
        proba.append(p[0]*p[1])
        proba.append(p[0]*p[1])
    for i in range(len(bigramme)):
        mdp.append(bigramme.letters.iloc[i]+bigramme.letters.iloc[i])
        proba.append(bigramme.frequency.iloc[i]+bigramme.frequency.iloc[i])
    return pd.DataFrame(dict(mdp=mdp, frequency=proba)).sort_values(by='frequency', ascending=False, ignore_index=True)

df_dictionary = generate_dictionary()

In [38]:
# Attaques sur 1000 mots de passes

nb_trial = 1000
vec_nb_trials = np.full(nb_trial, -1)
for i in range(nb_trial):
    passwrd = get_passwd()
    try:
        index = df_dictionary[df_dictionary['mdp'] ==passwrd].index.tolist()[0]
        vec_nb_trials[i] = index
    except:
        print(passwrd)

In [40]:
print(f"En moyenne, il y a {np.mean(vec_nb_trials)} appels.")

En moyenne, il y a 13581.98 appels.


In [42]:
minGBigramme/np.mean(vec_nb_trials)

21812.354067782537

## Conclusions 

- Définir des bonnes pratiques pour le défenseur, i.e. la personne cherchant à concevoir un système de génération de mots de passe ? 
L'entropie étant maximale dans le cas d'un tirage uniforme, c'est le meilleur moyen de générer un mot de passe robuste. Il faut ensuite travailler avec la longueur du mot de passe. Une longueur de 8 lettres n'est pas suffisant car le mot de passe est crackable en environ 13h. En passant à 12 lettres nous arrivons à 1000 ans ce qui rend le mot de passe non crackable. En rajoutant les lettres majuscules, les chiffres et les caractères spéciaux, on obtient un mot de passe parfait bien mieux que le mot de passe caramel par exemple.

- Définir des bonnes pratiques pour l'attaquant, i.e. la personne essayant de trouver le mot de passe ?
  Pour réduire le nombre d'appels avant de décoder un mot de passe, l'attanquant à intérêt à exploiter les aprioris que l'on peut faire sur un mot de passe à savoir  : La plupart des gens vont utiliser des mots de passe facile à retenir donc des mots de leur langue.
  Construire sa recherche de mot de passe autour de cet apriori probabiliste permet de tester en priorité les combinaisons les plus probables.
  (Dans le cadre de ce TP nous nous limitons à l'association de lettres)



## Un peu de lecture
Cet article montre comment des hackers, à partir de leaks de bases de mots de passes, peuvent rapidement arriver à trouver le votre:
https://arstechnica.com/information-technology/2013/05/how-crackers-make-minced-meat-out-of-your-passwords/
